In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers.initializers import variance_scaling_initializer

import skimage.io as io
import numpy as np
from matplotlib import pyplot as plt

import deepdish as dd
from os.path import join, dirname

import os.path as osp
import os
import sys
import pickle
curr_path = osp.dirname(os.getcwd())
model_dir = osp.join(curr_path, '..', 'models')

smpl_model_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\models\neutral_smpl_with_cocoplus_reg.pkl'
smpl_face_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\src\tf_smpl\smpl_faces.npy'

from src.batch_lbs import batch_rodrigues
from src.batch_smpl import SMPL
from src.projection import batch_orth_proj_idrot
from src.util import image as img_util

In [2]:
with open(smpl_model_path, 'rb') as f:
    dd = pickle.load(f, encoding='latin1')

In [3]:
[k + ': ' + str(np.array(dd[k]).shape) for k in dd.keys()]

['J: (24, 3)',
 'weights_prior: (6890, 24)',
 'posedirs: (6890, 3, 207)',
 'bs_style: ()',
 'v_template: (6890, 3)',
 'cocoplus_regressor: ()',
 'J_regressor_prior: ()',
 'f: (13776, 3)',
 'J_regressor: ()',
 'kintree_table: (2, 24)',
 'weights: (6890, 24)',
 'bs_type: ()',
 'shapedirs: (6890, 3, 10)']

In [6]:
# len(np.unique(dd['weights_prior'])) / (dd['weights_prior'].shape[0] * dd['weights_prior'].shape[1] ) * 100 

In [8]:
print(dd['cocoplus_regressor'].T.todense().shape)
print(dd['J_regressor_prior'].T.todense().shape)
print(dd['J_regressor'].T.todense().shape)

(6890, 19)
(6890, 24)
(6890, 24)


In [9]:
dd['bs_type']

'lrotmin'

In [ ]:
def preprocess_image(img_path, json_path=None):
    img = io.imread(img_path)
    if img.shape[2] == 4:
        img = img[:, :, :3]

    if json_path is None:
        if np.max(img.shape[:2]) != 224:
            print('Resizing so the max image size is %d..' % 224)
            scale = (float(224) / np.max(img.shape[:2]))
        else:
            scale = 1.
        center = np.round(np.array(img.shape[:2]) / 2).astype(int)
        # image center in (x,y)
        center = center[::-1]
    else:
        scale, center = op_util.get_bbox(json_path)

    crop, proc_param = img_util.scale_and_crop(img, scale, center, 224)

    # Normalize image to [-1, 1]
    crop = 2 * ((crop / 255.) - 0.5)

    return crop, proc_param, img

In [ ]:
img_path = 'data/im1963.jpg'
json_path = None
input_img, proc_param, img = preprocess_image(img_path, json_path)
input_img = np.expand_dims(input_img, 0)
input_img = np.float32(input_img)

In [ ]:
def load_mean_param():
    mean = np.zeros((1, 85))
    # Initialize scale at 0.9
    mean[0, 0] = 0.9
    mean_path = join(
        dirname(smpl_model_path), 'neutral_smpl_mean_params.h5')
    mean_vals = dd.io.load(mean_path)

    mean_pose = mean_vals['pose']
    # Ignore the global rotation.
    mean_pose[:3] = 0.
    mean_shape = mean_vals['shape']

    # This initializes the global pose to be up-right when projected
    mean_pose[0] = np.pi

    mean[0, 3:] = np.hstack((mean_pose, mean_shape))
    mean = tf.constant(mean, tf.float32)
    mean_var = tf.Variable(
        mean, name="mean_param", dtype=tf.float32, trainable=True)
#     E_var.append(mean_var)
    init_mean = tf.tile(mean_var, [1, 1])
    return init_mean

In [ ]:
use_3d_label = True
num_stage = 3
total_params = 85
num_cam = 3
num_theta = 72
batch_size = 1

In [ ]:
smpl = SMPL(smpl_model_path)
# verts, Js, pred_Rs = smpl(shapes, poses, get_skin=True)

In [ ]:
mean = np.zeros((1, 85))
# Initialize scale at 0.9
mean[0, 0] = 0.9
mean_path = join(
    dirname(smpl_model_path), 'neutral_smpl_mean_params.h5')
mean_vals = dd.io.load(mean_path)

mean_pose = mean_vals['pose']
# Ignore the global rotation.
mean_pose[:3] = 0.
mean_shape = mean_vals['shape']

# This initializes the global pose to be up-right when projected
mean_pose[0] = np.pi

mean[0, 3:] = np.hstack((mean_pose, mean_shape))
mean = tf.constant(mean, tf.float32)
mean_var = tf.Variable(
    mean, name="mean_param", dtype=tf.float32, trainable=True)
init_mean = tf.tile(mean_var, [1, 1])